# Alexander Toledo
## Analisis Exploratorio de Datos (EDA)
### Dataset: Customer Support on Twitter

In [2]:
import pandas as pd
import numpy as np

In [24]:
# Guardamos la data en un df
df = pd.read_csv('twcs.csv')
df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


## Contexto de las variables
### tweet_id
- A unique, anonymized ID for the Tweet. Referenced by response_tweet_id and in_response_to_tweet_id.
- Identificador único y anonimizado del tweet. Es referenciado por las variables response_tweet_id y in_response_to_tweet_id.
### author_id
- A unique, anonymized user ID. @s in the dataset have been replaced with their associated anonymized user ID.
- Identificador único y anonimizado del usuario. Las menciones (@) en el dataset fueron reemplazadas por su identificador de usuario anonimizado. 
### inbound
- Whether the tweet is "inbound" to a company doing customer support on Twitter. This feature is useful when re-organizing data for training conversational models.
- Indica si el tweet es una interacción “entrante” hacia una empresa que realiza atención al cliente en Twitter. Esta variable es útil para reorganizar los datos, por ejemplo, al entrenar modelos conversacionales.
### created_at
- Date and time when the tweet was sent.
- Fecha y Hora de cuando el tweet fue enviado.
### text
- Tweet content. Sensitive information like phone numbers and email addresses are replaced with mask values like "__email __".
- Contenido del tweet.
Información sensible como números de teléfono o correos electrónicos fue reemplazada por valores enmascarados, como "__email __". 
### response_tweet_id
- IDs of tweets that are responses to this tweet, comma-separated.
- Identificadores de los tweets que son respuestas a este tweet, separados por comas.
### in_response_to_tweet_id
- ID of the tweet this tweet is in response to, if any.
- Identificador del tweet al cual este tweet responde, si corresponde.

In [25]:
df.shape

(2811774, 7)

#### Dimension del Dataset
- Tenemos una data de 2.811.744 filas y 7 columnas

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2811774 entries, 0 to 2811773
Data columns (total 7 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tweet_id                 int64  
 1   author_id                object 
 2   inbound                  bool   
 3   created_at               object 
 4   text                     object 
 5   response_tweet_id        object 
 6   in_response_to_tweet_id  float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 131.4+ MB


#### Tipos de datos
- tweet_id es del tipo entero, y al ser un ID idealmente debe ser unico.
- author_id es del tipo object (string), porque al ser un ID de usuario anonimizado es alfanumerico.
- inbound es del tipo boolean, por lo que, es una varible con dos opciones.
- created_at es del tipo object (string), pero es una fecha, por lo tanto esta variable debe ser corregida para poder analizar los tiempos de respuesta.
- text es del tipo object (string), correcto al ser un texto.
- response_tweet_id es del tipo object, al hacer referencia a 0, 1 o más IDs y al estar separados por coma, los ID's se identifican como strings.
- in_response_to_tweet_id es del tipo float, al hacer referencia al tweet_id que es un int, esta variables tambien debiese ser int, para poder hacer joins con tweet_id.

In [27]:
# Transformamos la variables created_at de object (string) a datetime.
df['created_at'] = pd.to_datetime(
    df['created_at'],
    format = "%a %b %d %H:%M:%S %z %Y"
)
df["created_at"].dtype

datetime64[ns, UTC]

In [28]:
# Transformamos la variables in_response_to_tweet_id de float a int
df['in_response_to_tweet_id'] = df['in_response_to_tweet_id'].astype('Int64')
df['in_response_to_tweet_id'].dtype

Int64Dtype()

In [29]:
df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,2017-10-31 22:10:47+00:00,@115712 I understand. I would like to assist y...,2,3
1,2,115712,True,2017-10-31 22:11:45+00:00,@sprintcare and how do you propose we do that,NaN,1
2,3,115712,True,2017-10-31 22:08:27+00:00,@sprintcare I have sent several private messag...,1,4
3,4,sprintcare,False,2017-10-31 21:54:49+00:00,@115712 Please send us a Private Message so th...,3,5
4,5,115712,True,2017-10-31 21:49:35+00:00,@sprintcare I did.,4,6


In [30]:
df['tweet_id'].nunique()

2811774

In [31]:
df['author_id'].nunique()

702777

In [32]:
df['inbound'].unique()

array([False,  True])

In [35]:
df['response_tweet_id'].nunique()

1771145

In [38]:
df['in_response_to_tweet_id'].nunique()

1774822

#### Valores unicos
- Hay 2.811.774 tweet_id unicos, por lo tanto, no deberia existir ningun tweet repetido.
- Hay 702.777 author_id unicos, considerando lo anterior, hay más tweets que autores.
- En inbound solo se puede tener False y True, lo que significa que el dato tipo bool esta correcto.

In [33]:
nulos = df.isna().sum()
nulos

tweet_id                         0
author_id                        0
inbound                          0
created_at                       0
text                             0
response_tweet_id          1040629
in_response_to_tweet_id     794335
dtype: int64

#### Analisis nulos
- Lo nulos presentes se justifican debido a que no todos los tweets tienen respuestas asociadas y ademas no todos los tweets son respuestas a otros tweets, aquellos in_response_to_tweet_id que son nulos, son aquellos que inician un hilo, por lo tanto, existen 1.040.629, de tweets iniciales.
- Debido a esto, no se limpiaran estos valores y se trabajara con esos nulos como información relevante para el analisis.

In [34]:
duplicados = df.duplicated().sum()
duplicados

np.int64(0)

#### Analisis duplicados
- No hay filas duplicadas, asi que no hay informacion repetida.

## EDA Checklist
- Dimensión del dataset __R__
- Tipos de datos __R__
- Valores nulos y duplicados __R__
- Distribuciones básicas
- Valores únicos __R__
- Outliers
- Calidad del texto
- Consistencia lógica
- Cobertura temporal